In [ ]:
%%writefile K-means_CPU.cu

#include <iostream>
#include <random>
#include <fstream>
#include <sstream>
#include <cmath>
#include <limits>
#include <chrono>

using namespace std;


// Размерности массивов
const int size_mass1 = 10000;
const int size_mass2 = 100000;
const int size_mass3 = 1000000;

const int dim = 3; // размерность пространства
const int k = 3; // количество кластеров


// Пути к файлам с данными
string file1 = "my_mass10000.txt";     // для массива (10000, 3)
string file2 = "my_mass100000.txt";    // для массива (100000, 3)


// Функция для считывания из файлов данных с размерностями (10000, 3) и (100000, 3)
void readFromFile(float** data, int size, string fileName) {
    string filename = fileName;
    ifstream file(filename);
    if (file.is_open()) {
        string line;
        int i = 0;
        while (getline(file, line) && i < size) {
            istringstream iss(line);
            float x, y, z;
            if (iss >> x >> y >> z) {
                data[i][0] = x;
                data[i][1] = y;
                data[i][2] = z;
                i++;
            }
        }
        file.close();
    }
    else {
        cerr << "Error opening file: " << filename << endl;
    }
}


// Функция для считывания из файлов данных с размерностью (1000000, 3)
void readFromFile(float** data, int size, int part_number) {
    string filename = "my_mass1000000_part_" + to_string(part_number) + ".txt";
    ifstream file(filename);
    if (file.is_open()) {
        string line;
        int i = 0;
        while (getline(file, line) && i < size) {
            istringstream iss(line);
            float x, y, z;
            if (iss >> x >> y >> z) {
                data[i][0] = x;
                data[i][1] = y;
                data[i][2] = z;
                i++;
            }
        }
        file.close();
    }
    else {
        cerr << "Error opening file: " << filename << endl;
    }
}



void startingCenters(float** centers, float** data, int k) {

    float minValues[dim] = { 0.0f };
    float maxValues[dim] = { 0.0f };

    for (int i = 0; i < dim; i++) {
        minValues[i] = numeric_limits<float>::max();
        maxValues[i] = numeric_limits<float>::lowest();
    }

    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            centers[i][j] = 0;
        }
    }

    for (int i = 0; i < dim; i++) {
        for (int j = 0; j < k; j++) {
            minValues[i] = min(minValues[i], data[j][i]);
            maxValues[i] = max(maxValues[i], data[j][i]);
        }
    }

    random_device rd;
    mt19937 gen(rd());

    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            uniform_real_distribution<> dis(minValues[j], maxValues[j]);
            centers[i][j] = dis(gen);
        }
    }
}





// Функция для заполнения кластеров: точки, центры, кол-во точек, двум масс расстояний, массив индексов кластеров
void clusterDistribution(float** data, float** centers, int size_mass, float** distances, int* cluster_indexes, int k) {

    // Вычисление евклидовых расстояний от каждой точки до каждого центра
    for (int point = 0; point < size_mass; point++) {
        for (int num_center = 0; num_center < k; num_center++) {
            float sum = 0.0f;
            for (int coord = 0; coord < dim; coord++) {
                sum += pow((data[point][coord] - centers[num_center][coord]), 2);
            }
            float dist = sqrt(sum);
            distances[point][num_center] = dist;
        }
    }

    // Индекс минимального расстояния - это индекс кластера, к которому определяется точка
    for (int point = 0; point < size_mass; point++) {
        int min_index = 0;
        float min_distance = distances[point][0];  // предполагается, что первое расстояние минимальное


        // сравнение дистанций, нахождение минимальной
        for (int dist_id = 1; dist_id < k; dist_id++) {
            if (distances[point][dist_id] < min_distance) {
                min_distance = distances[point][dist_id];
                min_index = dist_id;
            }
        }

        // сохраняется индекс центра с минимальным расстояниянием для текущей точки
        cluster_indexes[point] = min_index;

        // cluster_indexes содержит номера кластеров для индексов каждой точки
    }
}




// Функция для пересчёта центров кластеров
void recalculationCenters(float** data, int size_mass, int* cluster_indexes, float** centers, int k) {

    // массив для подсчёта входящих индексов
    int* count_cluster_i = new int[k]();

    for (int i = 0; i < size_mass; i++) {
        int cluster_id = cluster_indexes[i];
        count_cluster_i[cluster_id]++;
    }

    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            float sum = 0.0f;
            for (int point = 0; point < size_mass; point++) {
                if (cluster_indexes[point] == i) {
                    sum += data[point][j];
                }
            }
            if (count_cluster_i[i] != 0) {
                centers[i][j] = sum / count_cluster_i[i];
            }
            else {
                k = k - 1;
                continue;
            }
        }
    }

    delete[] count_cluster_i;
}



// Функция для проверки сходимости алгоритма
bool converged(float** new_centers, float** old_centers, float epsilon, int k) {
    // Проверка сходимости по каждой компоненте центров
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            // Если разница между новым и предыдущим центром больше epsilon, алгоритм не сошелся
            if (fabs(new_centers[i][j] - old_centers[i][j]) >= epsilon) {
                return false;
            }
        }
    }
    return true; // Если ни одна разница не превышает epsilon, алгоритм считается сойденным
}



void k_means(float** data, int size_mass, int* cluster_indexes, float** centers, float** oldCenters, float** distances, int max_iterations, float epsilon) {


    // Инициализация начальных центров кластеров
    startingCenters(centers, data, k);

    // Копирование текущих центров в массив oldCenters
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            oldCenters[i][j] = centers[i][j];
        }
    }

    int iteration = 0;
    while (iteration < max_iterations) {

        // Заполнение кластеров
        clusterDistribution(data, centers, size_mass, distances, cluster_indexes, k);

        // Пересчёт новых центров кластеров
        recalculationCenters(data, size_mass, cluster_indexes, centers, k);


        // Условие остановки алгоритма
 //       if (iteration > 0 && converged(centers, oldCenters, epsilon, k)) {
 //           cout << "The algorithm converged on iteration " << iteration << "." << endl << endl;
 //           break;
 //       }

        // Обновляем oldCenters
        for (int i = 0; i < k; i++) {
            for (int j = 0; j < dim; j++) {
                oldCenters[i][j] = centers[i][j];
            }
        }

        iteration++;
    }

}


int main() {

    const int max_iterations = 10;        // Максимальное количество итераций
    const float epsilon = 0.000001;        // Пороговое значение для остановки

    const int numIterations = 10;          // количество замеров времени



    // Выделение памяти под массивы

    // Данные с точками
    float** myArray1 = new float* [size_mass1];
    for (int i = 0; i < size_mass1; i++) {
        myArray1[i] = new float[dim];
    }

    float** myArray2 = new float* [size_mass2];
    for (int i = 0; i < size_mass2; i++) {
        myArray2[i] = new float[dim];
    }

    float** myArray3 = new float* [size_mass3];
    for (int i = 0; i < size_mass3; i++) {
        myArray3[i] = new float[dim];
    }

    // Центры
    float** centers1 = new float* [k];
    for (int i = 0; i < k; i++) {
        centers1[i] = new float[dim];
    }

    float** centers2 = new float* [k];
    for (int i = 0; i < k; i++) {
        centers2[i] = new float[dim];
    }
    float** centers3 = new float* [k];
    for (int i = 0; i < k; i++) {
        centers3[i] = new float[dim];
    }


    // Старые центры
    float** oldCenters1 = new float* [k];
    for (int i = 0; i < k; i++) {
        oldCenters1[i] = new float[dim];
    }

    float** oldCenters2 = new float* [k];
    for (int i = 0; i < k; i++) {
        oldCenters2[i] = new float[dim];
    }

    float** oldCenters3 = new float* [k];
    for (int i = 0; i < k; i++) {
        oldCenters3[i] = new float[dim];
    }



    // Для вычисления евклидовых расстояний
    float** distances1 = new float* [size_mass1];
    for (int i = 0; i < size_mass1; i++) {
        distances1[i] = new float[k];
    }

    float** distances2 = new float* [size_mass2];
    for (int i = 0; i < size_mass2; i++) {
        distances2[i] = new float[k];
    }

    float** distances3 = new float* [size_mass3];
    for (int i = 0; i < size_mass3; i++) {
        distances3[i] = new float[k];
    }


    // Массивы для хранения индексов кластеров
    int* cluster_indexes1 = new int[size_mass1];
    int* cluster_indexes2 = new int[size_mass2];
    int* cluster_indexes3 = new int[size_mass3];


    // Считывание данных (10000, 3) из файла и запись в массив
    readFromFile(myArray1, size_mass1, file1);

    // Считывание данных (100000, 3) из файла и запись в массив
    readFromFile(myArray2, size_mass2, file2);

    // Считывание данных (1000000, 3) из файла и запись в массив
    for (int i = 0; i <= 9; i++) {
        readFromFile(myArray3 + i * 100000, size_mass3 / 10, i);
    }


    //cout << "K-means algorithm. Start of program timing." << endl << endl;


    // Замер времени для массива размерностью (10000, 3)
    //cout << "Start timing mass (10000,3)" << endl << endl;
    double total_duration1 = 0.0f;
    for (int i = 0; i < numIterations; i++) {
    //    cout << "timing iteration: " << i << endl << endl;
        auto start1 = chrono::steady_clock::now();
        k_means(myArray1, size_mass1, cluster_indexes1, centers1, oldCenters1, distances1, max_iterations, epsilon);
        auto end1 = chrono::steady_clock::now();
        auto duration1 = chrono::duration_cast<chrono::milliseconds>(end1 - start1).count();
        total_duration1 += duration1;
    }
    //среднее время из 10 замеров
    double average_duration1 = total_duration1 / numIterations;
    //cout << "CPU C++ (10000,3): " << average_duration1 << " ms." << endl << endl;



    // Замер времени для массива размерностью (100000, 3)
    //cout << "Start timing mass (100000,3)" << endl << endl;
    double total_duration2 = 0.0f;
    for (int i = 0; i < numIterations; i++) {
    //    cout << "timing iteration: " << i << endl << endl;
        auto start2 = chrono::steady_clock::now();
        k_means(myArray2, size_mass2, cluster_indexes2, centers2, oldCenters2, distances2, max_iterations, epsilon);
        auto end2 = chrono::steady_clock::now();
        auto duration2 = chrono::duration_cast<chrono::milliseconds>(end2 - start2).count();
        total_duration2 += duration2;
    }
    //среднее время из 10 замеров
    double average_duration2 = total_duration2 / numIterations;
    //cout << "CPU C++ (100000,3): " << average_duration2 << " ms." << endl << endl;



    // Замер времени для массива размерностью (1000000, 3)
    //cout << "Start timing mass (1000000,3)" << endl << endl;
    double total_duration3 = 0.0f;
    for (int i = 0; i < numIterations; i++) {
    //    cout << "timing iteration: " << i << endl << endl;

        auto start3 = chrono::steady_clock::now();
        k_means(myArray3, size_mass3, cluster_indexes3, centers3, oldCenters3, distances3, max_iterations, epsilon);
        auto end3 = chrono::steady_clock::now();
        auto duration3 = chrono::duration_cast<chrono::milliseconds>(end3 - start3).count();
        total_duration3 += duration3;
    }
    //среднее время из 10 замеров
    double average_duration3 = total_duration3 / numIterations;
    //cout << "CPU C++ (1000000,3): " << average_duration3 << " ms." << endl << endl;


    // Итоговый вывод результатов
    cout << endl << endl << "Results:" << endl << endl;
    cout << "K-means algorithm. Program running time averaged over " << numIterations << " iterations." << endl << endl;

    cout << "CPU C++ (10000,3): " << average_duration1 << " ms." << endl;
    cout << "CPU C++ (100000,3): " << average_duration2 << " ms." << endl;
    cout << "CPU C++ (1000000,3): " << average_duration3 << " ms." << endl;



    // Освобождение памяти

    // удаление массивов для хранения данных с точками
    for (int i = 0; i < size_mass1; i++) {
        delete[] myArray1[i];
    }
    delete[] myArray1;

    for (int i = 0; i < size_mass2; i++) {
        delete[] myArray2[i];
    }
    delete[] myArray2;

    for (int i = 0; i < size_mass3; i++) {
        delete[] myArray3[i];
    }
    delete[] myArray3;


    // Удаление центров
    for (int i = 0; i < k; i++) {
        delete[] centers1[i];
    }
    delete[] centers1;

    for (int i = 0; i < k; i++) {
        delete[] centers2[i];
    }
    delete[] centers2;

    for (int i = 0; i < k; i++) {
        delete[] centers3[i];
    }
    delete[] centers3;


    for (int i = 0; i < k; i++) {
        delete[] oldCenters1[i];
    }
    delete[] oldCenters1;

    for (int i = 0; i < k; i++) {
        delete[] oldCenters2[i];
    }
    delete[] oldCenters2;

    for (int i = 0; i < k; i++) {
        delete[] oldCenters3[i];
    }
    delete[] oldCenters3;


    // Удаление массивов для вычисления евклидовых расстояний
    for (int i = 0; i < k; i++) {
        delete[] distances1[i];
    }
    delete[] distances1;

    for (int i = 0; i < k; i++) {
        delete[] distances2[i];
    }
    delete[] distances2;

    for (int i = 0; i < k; i++) {
        delete[] distances3[i];
    }
    delete[] distances3;



    // Удаление массива индексов кластеров
    delete[] cluster_indexes1;
    delete[] cluster_indexes2;
    delete[] cluster_indexes3;


    return 0;
}


Writing K-means_CPU.cu


In [ ]:
!nvcc K-means_CPU.cu -o K-means_CPU
# компилляция

In [ ]:
# Запуск
!./K-means_CPU



Results:

K-means algorithm. Program running time averaged over 10 iterations.

CPU C++ (10000,3): 37.6 ms.
CPU C++ (100000,3): 379.5 ms.
CPU C++ (1000000,3): 4071.2 ms.


In [ ]:
%%writefile C-means_CPU.cu

#include <iostream>
#include <random>
#include <fstream>
#include <sstream>
#include <cmath>
#include <limits>
#include <chrono>

using namespace std;


// Размерности массивов
const int size_mass1 = 10000;
const int size_mass2 = 100000;
const int size_mass3 = 1000000;

const int dim = 3; // размерность пространства
const int k = 3; // количество кластеров


// Пути к файлам с данными
string file1 = "my_mass10000.txt";     // для массива (10000, 3)
string file2 = "my_mass100000.txt";    // для массива (100000, 3)


// Функция для считывания данных из файлов с размерностями (10000, 3) и (100000, 3)
void readFromFile(float** data, int size, string fileName) {
    string filename = fileName;
    ifstream file(filename);
    if (file.is_open()) {
        string line;
        int i = 0;
        while (getline(file, line) && i < size) {
            istringstream iss(line);
            float x, y, z;
            if (iss >> x >> y >> z) {
                data[i][0] = x;
                data[i][1] = y;
                data[i][2] = z;
                i++;
            }
        }
        file.close();
    }
    else {
        cerr << "Error opening file: " << filename << endl;
    }
}


// Функция для считывания данных из файла размерностью (1000000, 3)
void readFromFile(float** data, int size, int part_number) {
    string filename = "my_mass1000000_part_" + to_string(part_number) + ".txt";
    ifstream file(filename);
    if (file.is_open()) {
        string line;
        int i = 0;
        while (getline(file, line) && i < size) {
            istringstream iss(line);
            float x, y, z;
            if (iss >> x >> y >> z) {
                data[i][0] = x;
                data[i][1] = y;
                data[i][2] = z;
                i++;
            }
        }
        file.close();
    }
    else {
        cerr << "Error opening file: " << filename << endl;
    }
}



void startingCenters(float** centers, float** data, int k) {

    float minValues[dim] = { 0.0f };
    float maxValues[dim] = { 0.0f };

    for (int i = 0; i < dim; i++) {
        minValues[i] = numeric_limits<float>::max();
        maxValues[i] = numeric_limits<float>::lowest();
    }

    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            centers[i][j] = 0;
        }
    }

    for (int i = 0; i < dim; i++) {
        for (int j = 0; j < k; j++) {
            minValues[i] = min(minValues[i], data[j][i]);
            maxValues[i] = max(maxValues[i], data[j][i]);
        }
    }

    random_device rd;
    mt19937 gen(rd());

    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            uniform_real_distribution<> dis(minValues[j], maxValues[j]);
            centers[i][j] = dis(gen);
        }
    }
}



//  пересчёт матрицы степеней принадлежности на основе вычисленных центроид
void updateMembership(float** dataPoints, int size_mass, float** centers, float** distances, float** membership, int k, int m) {

    // Вычисление евклидовых расстояний от каждой точки до каждого центра
    for (int point = 0; point < size_mass; point++) {
        for (int num_center = 0; num_center < k; num_center++) {
            float sum = 0.0f;
            for (int coord = 0; coord < dim; coord++) {
                sum += pow((dataPoints[point][coord] - centers[num_center][coord]), 2);
            }
            float dist = sqrt(sum);
            distances[point][num_center] = dist;
        }
    }

    // Вычисление степеней принадлежности для каждой точки
    for (int i = 0; i < size_mass; i++) {
        for (int j = 0; j < k; j++) {
            float sumDistances = 0.0f;
            for (int c = 0; c < k; c++) {
                float distancesRatio = distances[i][j] / distances[i][c];
                sumDistances += pow(distancesRatio, (2.0f / (m - 1)));
            }
            membership[i][j] = 1.0f / sumDistances;
        }
    }
}



// функция для пересчёта центров
void updateCenters(float** data, int mass_size, float** centers, float** membership, int m) {


    // пересчёт центров
    for (int num_center = 0; num_center < k; num_center++) {


        // возведение элементов матрицы в степень
        for (int i = 0; i < mass_size; i++) {
            for (int j = 0; j < k; j++) {
                membership[i][j] = pow(membership[i][j], m);
            }
        }

        float numerator[dim] = { 0.0f };
        float denominator = 0.0f;

        // вычисление числителя и знаменателя
        for (int i = 0; i < mass_size; i++) {
            for (int j = 0; j < dim; j++) {
                numerator[j] += membership[i][num_center] * data[i][j];
            }
            denominator += membership[i][num_center];
        }

        // пересчёт центра
        for (int j = 0; j < dim; j++) {
            centers[num_center][j] = numerator[j] / denominator;
        }

    }

}



// Функция для проверки сходимости алгоритма
bool converged(float** new_centers, float** old_centers, float epsilon, int k) {
    // Проверка сходимости по каждой компоненте центров
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            // Если разница между новым и предыдущим центром больше epsilon, алгоритм не сошелся
            if (fabs(new_centers[i][j] - old_centers[i][j]) >= epsilon) {
                return false;
            }
        }
    }
    return true; // Если ни одна разница не превышает epsilon, алгоритм считается сойденным
}



// Основная функция алгоритма C-Means
void c_means(float** data, int mass_size, float** centers, float** oldCenters, float** membership, float** distances, int max_iterations, int m, float epsilon) {

    // Инициализация начальных центроид
    startingCenters(centers, data, k);

    // Копирование текущих центров в массив oldCenters
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            oldCenters[i][j] = centers[i][j];
        }
    }

    // Итерационный процесс
    int iteration = 0;
    while (iteration < max_iterations) {

        // Рассчет матрицы принадлежности на основе текущих центров
        updateMembership(data, mass_size, centers, distances, membership, k, m);

        // Пересчет центров
        updateCenters(data, mass_size, centers, membership, m);

        // Условие остановки алгоритма
//        if (converged(centers, oldCenters, epsilon, k)) {
//            cout << "The algorithm converged on iteration " << iteration << endl;
//            break;
//        }

        // Обновление центров
        for (int i = 0; i < k; i++) {
            for (int j = 0; j < dim; j++) {
                oldCenters[i][j] = centers[i][j];
            }
        }

        iteration++;
    }
}



int main() {

    const int m = 2; // Экспоненциальный вес
    const int max_iterations = 10; // Максимальное количество итераций
    const float epsilon = 0.0001; // Пороговое значение для остановки

    const int numIterations = 10; // количество замеров времени


    // Выделение памяти под массивы

    // Данные с точками
    float** myArray1 = new float* [size_mass1];
    for (int i = 0; i < size_mass1; i++) {
        myArray1[i] = new float[dim];
    }

    float** myArray2 = new float* [size_mass2];
    for (int i = 0; i < size_mass2; i++) {
        myArray2[i] = new float[dim];
    }

    float** myArray3 = new float* [size_mass3];
    for (int i = 0; i < size_mass3; i++) {
        myArray3[i] = new float[dim];
    }

    // Центры
    float** centers1 = new float* [k];
    for (int i = 0; i < k; i++) {
        centers1[i] = new float[dim];
    }

    float** centers2 = new float* [k];
    for (int i = 0; i < k; i++) {
        centers2[i] = new float[dim];
    }
    float** centers3 = new float* [k];
    for (int i = 0; i < k; i++) {
        centers3[i] = new float[dim];
    }


    // Старые центры
    float** oldCenters1 = new float* [k];
    for (int i = 0; i < k; i++) {
        oldCenters1[i] = new float[dim];
    }

    float** oldCenters2 = new float* [k];
    for (int i = 0; i < k; i++) {
        oldCenters2[i] = new float[dim];
    }

    float** oldCenters3 = new float* [k];
    for (int i = 0; i < k; i++) {
        oldCenters3[i] = new float[dim];
    }

    // Матрицы степеней принадлежности
    float** membership1 = new float* [size_mass1];
    for (int i = 0; i < size_mass1; i++) {
        membership1[i] = new float[k];
    }

    float** membership2 = new float* [size_mass2];
    for (int i = 0; i < size_mass2; i++) {
        membership2[i] = new float[k];
    }

    float** membership3 = new float* [size_mass3];
    for (int i = 0; i < size_mass3; i++) {
        membership3[i] = new float[k];
    }


    // Для вычисления евклидовых расстояний
    float** distances1 = new float* [size_mass1];
    for (int i = 0; i < size_mass1; i++) {
        distances1[i] = new float[k];
    }

    float** distances2 = new float* [size_mass2];
    for (int i = 0; i < size_mass2; i++) {
        distances2[i] = new float[k];
    }

    float** distances3 = new float* [size_mass3];
    for (int i = 0; i < size_mass3; i++) {
        distances3[i] = new float[k];
    }




    // Считывание данных (10000, 3) из файла и запись в массив
    readFromFile(myArray1, size_mass1, file1);

    // Считывание данных (100000, 3) из файла и запись в массив
    readFromFile(myArray2, size_mass2, file2);

    // Считывание данных (1000000, 3) из файла и запись в массив
    for (int i = 0; i <= 9; i++) {
        readFromFile(myArray3 + i * 100000, size_mass3 / 10, i);
    }



    cout << "C-means algorithm. Start of program timing." << endl << endl;

    //cout << "Start timing mass (10000,3)" << endl << endl;

    // Замер времени для массива размерностью (10000, 3)
    double total_duration1 = 0.0f;
    for (int i = 0; i < numIterations; i++) {
    //    cout << "timing iteration: " << i << endl << endl;
        auto start1 = chrono::steady_clock::now();
        c_means(myArray1, size_mass1, centers1, oldCenters1, membership1, distances1, max_iterations, m, epsilon);
        auto end1 = chrono::steady_clock::now();
        auto duration1 = chrono::duration_cast<chrono::milliseconds>(end1 - start1).count();
        total_duration1 += duration1;
    }
    //среднее время из 10 замеров
    double average_duration1 = total_duration1 / numIterations;
    //cout << "CPU C++ (10000,3): " << average_duration1 << " ms." << endl << endl;

    //cout << "Start timing mass (100000,3)" << endl << endl;

    // Замер времени для массива размерностью (100000, 3)
    double total_duration2 = 0.0f;
    for (int i = 0; i < numIterations; i++) {
    //    cout << "timing iteration: " << i << endl << endl;
        auto start2 = chrono::steady_clock::now();
        c_means(myArray2, size_mass2, centers2, oldCenters2, membership2, distances2, max_iterations, m, epsilon);
        auto end2 = chrono::steady_clock::now();
        auto duration2 = chrono::duration_cast<chrono::milliseconds>(end2 - start2).count();
        total_duration2 += duration2;
    }
    //среднее время из 10 замеров
    double average_duration2 = total_duration2 / numIterations;
    //cout << "CPU C++ (100000,3): " << average_duration2 << " ms." << endl << endl;

    //cout << "Start timing mass (1000000,3)" << endl << endl;

    // Замер времени для массива размерностью (1000000, 3)
    double total_duration3 = 0.0f;
    for (int i = 0; i < numIterations; i++) {
    //    cout << "timing iteration: " << i << endl << endl;

        auto start3 = chrono::steady_clock::now();
        c_means(myArray3, size_mass3, centers3, oldCenters3, membership3, distances3, max_iterations, m, epsilon);
        auto end3 = chrono::steady_clock::now();
        auto duration3 = chrono::duration_cast<chrono::milliseconds>(end3 - start3).count();
        total_duration3 += duration3;
    }
    //среднее время из 10 замеров
    double average_duration3 = total_duration3 / numIterations;
    //cout << "CPU C++ (1000000,3): " << average_duration3 << " ms." << endl << endl;


    // Итоговый вывод результатов
    cout << endl << endl << "Results:" << endl << endl;
    cout << "C-means algorithm. Program running time averaged over " << numIterations << " iterations." << endl << endl;

    cout << "CPU C++ (10000,3): " << average_duration1 << " ms." << endl;
    cout << "CPU C++ (100000,3): " << average_duration2 << " ms." << endl;
    cout << "CPU C++ (1000000,3): " << average_duration3 << " ms." << endl;



    // Освобождение памяти

    // удаление массивов для хранения данных с точками
    for (int i = 0; i < size_mass1; ++i) {
        delete[] myArray1[i];
    }
    delete[] myArray1;

    for (int i = 0; i < size_mass2; ++i) {
        delete[] myArray2[i];
    }
    delete[] myArray2;

    for (int i = 0; i < size_mass3; ++i) {
        delete[] myArray3[i];
    }
    delete[] myArray3;

    // удаление массивов для хранения координат центров
    for (int i = 0; i < k; i++) {
        delete[] centers1[i];
    }
    delete[] centers1;

    for (int i = 0; i < k; i++) {
        delete[] centers2[i];
    }
    delete[] centers2;

    for (int i = 0; i < k; i++) {
        delete[] centers3[i];
    }
    delete[] centers3;


    for (int i = 0; i < k; i++) {
        delete[] oldCenters1[i];
    }
    delete[] oldCenters1;

    for (int i = 0; i < k; i++) {
        delete[] oldCenters2[i];
    }
    delete[] oldCenters2;

    for (int i = 0; i < k; i++) {
        delete[] oldCenters3[i];
    }
    delete[] oldCenters3;



    // удаление массивов для матриц степеней принадлежности
    for (int i = 0; i < size_mass1; i++) {
        delete[] membership1[i];
    }
    delete[] membership1;

    for (int i = 0; i < size_mass2; i++) {
        delete[] membership2[i];
    }
    delete[] membership2;

    for (int i = 0; i < size_mass3; i++) {
        delete[] membership3[i];
    }
    delete[] membership3;


    // удаление массивов для вычисления евклидовых расстояний
    for (int i = 0; i < k; i++) {
        delete[] distances1[i];
    }
    delete[] distances1;

    for (int i = 0; i < k; i++) {
        delete[] distances2[i];
    }
    delete[] distances2;

    for (int i = 0; i < k; i++) {
        delete[] distances3[i];
    }
    delete[] distances3;


    return 0;
}

Overwriting C-means_CPU.cu


In [ ]:
!nvcc C-means_CPU.cu -o C-means_CPU
# компилляция

In [ ]:
# Запуск
!./C-means_CPU

C-means algorithm. Start of program timing.



Results:

C-means algorithm. Program running time averaged over 10 iterations.

CPU C++ (10000,3): 84.2 ms.
CPU C++ (100000,3): 915.6 ms.
CPU C++ (1000000,3): 8938.6 ms.


In [ ]:
%%writefile K-means10000_CUDA.cu

#include <iostream>
#include <random>
#include <fstream>
#include <sstream>
#include <cmath>
#include <algorithm>
#include <limits>
#include <cuda_runtime.h>

using namespace std;

// Константы для замера времени
const int iterations = 100;
float totalMallocTime = 0.0f;
float totalKmTime = 0.0f;
float totalMemcpyTime = 0.0f;
float totalFreeTime = 0.0f;

// Константы и параметры
const int size_mass = 10000;   // размерность массива
const int dim = 3;             // размерность пространства
const int k = 3;               // количество кластеров
const int max_iterations = 10; // Максимальное количество итераций
// const float epsilon = 0.00001; // Пороговое значение для остановки

// Путь к файлу с данными
string file = "my_mass10000.txt";

// Функция для считывания из файла данных с размерностью (10000, 3)
void readFromFile(float* data, int size, string fileName) {
    ifstream file(fileName);
    if (file.is_open()) {
        string line;
        int i = 0;
        while (getline(file, line) && i < size) {
            istringstream iss(line);
            float x, y, z;
            if (iss >> x >> y >> z) {
                data[i * dim] = x;
                data[i * dim + 1] = y;
                data[i * dim + 2] = z;
                i++;
            }
        }
        file.close();
    } else {
        cerr << "Error opening file: " << fileName << endl;
    }
}

// Задание начальных центров на CPU
void startingCenters(float* centers, float* data) {
    float minValues[dim], maxValues[dim];
    for (int i = 0; i < dim; i++) {
        minValues[i] = numeric_limits<float>::max();
        maxValues[i] = numeric_limits<float>::lowest();
    }

    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            centers[i * dim + j] = 0;
        }
    }

    for (int i = 0; i < size_mass; i++) {
        for (int j = 0; j < dim; j++) {
            minValues[j] = min(minValues[j], data[i * dim + j]);
            maxValues[j] = max(maxValues[j], data[i * dim + j]);
        }
    }

    random_device rd;
    mt19937 gen(rd());

    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            uniform_real_distribution<> dis(minValues[j], maxValues[j]);
            centers[i * dim + j] = dis(gen);
        }
    }
}

// Объявление константной памяти на GPU
__constant__ int d_k;
__constant__ int d_dim;
__constant__ int d_size_mass;

// CUDA-ядро для заполнения кластеров
__global__ void clusterDistrCUDA(float* data, float* centers, float* distances, int* cluster_indexes) {
    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    // Работа всех нитей
    if (point < d_size_mass) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            float sum = 0.0f;
            for (int coord = 0; coord < d_dim; coord++) {
                sum += powf((data[point * d_dim + coord] - centers[num_center * d_dim + coord]), 2);
            }
            float dist = sqrtf(sum);
            distances[point * d_k + num_center] = dist;
        }

        int min_index = 0;
        float min_distance = distances[point * d_k];

        for (int dist_id = 1; dist_id < d_k; dist_id++) {
            if (distances[point * d_k + dist_id] < min_distance) {
                min_distance = distances[point * d_k + dist_id];
                min_index = dist_id;
            }
        }

        cluster_indexes[point] = min_index;
    }
}

// CUDA-ядро для обнуления счётчика и центров
__global__ void nullCentersCUDA(float* data, int* cluster_indexes, float* centers, int* count_cluster_i) {
    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    // Блок для одной нити, обнуление счётчика
    if (point == 0) {
        for (int i = 0; i < d_k; i++) {
            for (int j = 0; j < d_dim; j++) {
                centers[i * d_dim + j] = 0.0f;
            }
            count_cluster_i[i] = 0;
        }
    }
}

// CUDA-ядро для пересчёта центров
__global__ void recalculCentersCUDA(float* data, int* cluster_indexes, float* centers, int* count_cluster_i) {
    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    // Блок для всех нитей
    if (point < d_size_mass) {
        int cluster_id = cluster_indexes[point];
        atomicAdd(&count_cluster_i[cluster_id], 1);
        for (int j = 0; j < d_dim; j++) {
            // если точка относится к кластеру, её координаты суммируются
            atomicAdd(&centers[cluster_id * d_dim + j], data[point * d_dim + j]);
        }
    }
}

__global__ void coordCUDA(float* data, int* cluster_indexes, float* centers, int* count_cluster_i) {
    int point = blockIdx.x * blockDim.x + threadIdx.x;
    // блок для одной нити
    if (point == 0) {
        for (int i = 0; i < d_k; i++) {
            for (int j = 0; j < d_dim; j++) {
                if (count_cluster_i[i] != 0) {
                    // вычисление координаты
                    centers[i * d_dim + j] /= count_cluster_i[i];
                }
            }
        }
    }
}

void k_means(float* data, int* cluster_indexes, float* centers, float* oldCenters, float* distances, float* dev_data, float* dev_centers, float* dev_distances, int* dev_cluster_indexes, int* dev_count_cluster_i) {
    // Копирование констант на GPU
    cudaMemcpyToSymbol(d_k, &k, sizeof(int));
    cudaMemcpyToSymbol(d_dim, &dim, sizeof(int));
    cudaMemcpyToSymbol(d_size_mass, &size_mass, sizeof(int));

    // Инициализация центров на CPU
    startingCenters(centers, data);

    // Перенос начальных центров на GPU
    cudaMemcpy(dev_centers, centers, k * dim * sizeof(float), cudaMemcpyHostToDevice);

    // Сохранение начальных центров на CPU
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            oldCenters[i * dim + j] = centers[i * dim + j];
        }
    }

    int iteration = 0;
    while (iteration < max_iterations) {
        // Определение параметров для запуска ядер
        int blockSize = 256;
        int gridSize = (size_mass + blockSize - 1) / blockSize;

        // запуск ядра для заполнения кластеров
        clusterDistrCUDA<<<gridSize, blockSize>>>(dev_data, dev_centers, dev_distances, dev_cluster_indexes);
        cudaDeviceSynchronize();

        nullCentersCUDA<<<1, 1>>>(dev_data, dev_cluster_indexes, dev_centers, dev_count_cluster_i);
        cudaDeviceSynchronize();

        // запуск ядра для пересчёта центров
        recalculCentersCUDA<<<gridSize, blockSize>>>(dev_data, dev_cluster_indexes, dev_centers, dev_count_cluster_i);
        cudaDeviceSynchronize();

        coordCUDA<<<1, 1>>>(dev_data, dev_cluster_indexes, dev_centers, dev_count_cluster_i);
        cudaDeviceSynchronize();

        // копирование пересчитанных центров с GPU на CPU
        cudaMemcpy(centers, dev_centers, k * dim * sizeof(float), cudaMemcpyDeviceToHost);

        // проверка сходимости
        // if (iteration > 0 && converged(centers, oldCenters, epsilon, k)) {
        //     cout << "The algorithm converged on iteration " << iteration << "." << endl << endl;
        //     break;
        // }

        // сохранение вычисленных центров
        for (int i = 0; i < k; i++) {
            for (int j = 0; j < dim; j++) {
                oldCenters[i * dim + j] = centers[i * dim + j];
            }
        }

        iteration++;
    }

    // Копирование меток для точек с GPU на CPU
    cudaMemcpy(cluster_indexes, dev_cluster_indexes, size_mass * sizeof(int), cudaMemcpyDeviceToHost);
}

int main() {

    for (int iter = 0; iter < iterations; iter++) {
        // Выделение памяти на CPU
        float* myArray = new float[size_mass * dim];
        float* centers = new float[k * dim];
        float* oldCenters = new float[k * dim];
        float* distances = new float[size_mass * k];
        int* cluster_indexes = new int[size_mass];

        // -------------------------------------------------------------------------
        // замеры времени выделения памяти с усреднением по итерациям
        cudaEvent_t start1, stop1;
        cudaEventCreate(&start1);
        cudaEventCreate(&stop1);

        cudaEventRecord(start1);

        // Выделение памяти на GPU
        float *dev_data, *dev_centers, *dev_distances;
        int *dev_cluster_indexes, *dev_count_cluster_i;

        cudaMalloc((void**)&dev_data, size_mass * dim * sizeof(float));
        cudaMalloc((void**)&dev_centers, k * dim * sizeof(float));
        cudaMalloc((void**)&dev_distances, size_mass * k * sizeof(float));
        cudaMalloc((void**)&dev_cluster_indexes, size_mass * sizeof(int));
        cudaMalloc((void**)&dev_count_cluster_i, k * sizeof(int));

        cudaEventRecord(stop1);
        cudaEventSynchronize(stop1);
        float mallocTime;
        cudaEventElapsedTime(&mallocTime, start1, stop1);
        totalMallocTime += mallocTime;
        cudaEventDestroy(start1);
        cudaEventDestroy(stop1);
        // --------------------------------------------------------------------------

        // Считывание данных из файла и запись в массив
        readFromFile(myArray, size_mass, file);

        // __________________
        // замеры времени копирования данных на устройство с усреднением по итерациям
        cudaEvent_t start2, stop2;
        cudaEventCreate(&start2);
        cudaEventCreate(&stop2);

        cudaEventRecord(start2);

        // Копирование данных на устройство
        cudaMemcpy(dev_data, myArray, size_mass * dim * sizeof(float), cudaMemcpyHostToDevice);

        cudaEventRecord(stop2);
        cudaEventSynchronize(stop2);
        float memcpyTime;
        cudaEventElapsedTime(&memcpyTime, start2, stop2);
        totalMemcpyTime += memcpyTime;
        cudaEventDestroy(start2);
        cudaEventDestroy(stop2);
        // __________________

        // Запуск алгоритма
        // cout << "K-means algorithm. Mass (10000,3)." << endl << endl;

        // ...........................................................................
        // замеры времени работы алгоритма K-means с усреднением по итерациям
        cudaEvent_t start3, stop3;
        cudaEventCreate(&start3);
        cudaEventCreate(&stop3);

        cudaEventRecord(start3);

        k_means(myArray, cluster_indexes, centers, oldCenters, distances, dev_data, dev_centers, dev_distances, dev_cluster_indexes, dev_count_cluster_i);

        cudaEventRecord(stop3);
        cudaEventSynchronize(stop3);
        float kmTime;
        cudaEventElapsedTime(&kmTime, start3, stop3);
        totalKmTime += kmTime;
        cudaEventDestroy(start3);
        cudaEventDestroy(stop3);
        // ............................................................................

        // Освобождение памяти на CPU
        delete[] myArray;
        delete[] centers;
        delete[] oldCenters;
        delete[] distances;
        delete[] cluster_indexes;

        // ******************
        // замеры времени освобождения памяти на GPU с усреднением по итерациям
        cudaEvent_t start4, stop4;
        cudaEventCreate(&start4);
        cudaEventCreate(&stop4);

        cudaEventRecord(start4);

        // Освобождение памяти на GPU
        cudaFree(dev_data);
        cudaFree(dev_centers);
        cudaFree(dev_distances);
        cudaFree(dev_cluster_indexes);
        cudaFree(dev_count_cluster_i);

        cudaEventRecord(stop4);
        cudaEventSynchronize(stop4);
        float freeTime;
        cudaEventElapsedTime(&freeTime, start4, stop4);
        totalFreeTime += freeTime;
        cudaEventDestroy(start4);
        cudaEventDestroy(stop4);
        // ****************
    }

    cout << "\nK_means. For mass 10000 points" << endl;
    cout << "GPU memory allocation time: " << totalMallocTime / iterations << " ms" << endl;
    cout << "Time to copy data from the host to the device: " << totalMemcpyTime / iterations << " ms" << endl;
    cout << "K-means: " << totalKmTime / iterations << " ms" << endl;
    cout << "Free GPU: " << totalFreeTime / iterations << " ms" << endl;
    cout << "Total: " << (totalMallocTime + totalMemcpyTime + totalKmTime + totalFreeTime) / iterations << " ms" << endl;

    return 0;
}

Overwriting K-means10000_CUDA.cu


In [ ]:
!nvcc K-means10000_CUDA.cu -o K-means10000_CUDA
# компилляция

In [ ]:
# Запуск
!./K-means10000_CUDA


K_means. For mass 10000 points
GPU memory allocation time: 0.128587 ms
Time to copy data from the host to the device: 0.0538922 ms
K-means: 2.14689 ms
Free GPU: 0.134365 ms
Total: 2.46373 ms


In [ ]:
%%writefile K-means100000_CUDA.cu

#include <iostream>
#include <random>
#include <fstream>
#include <sstream>
#include <cmath>
#include <algorithm>
#include <limits>
#include <cuda_runtime.h>

using namespace std;

// Константы для замера времени
const int iterations = 100;
float totalMallocTime = 0.0f;
float totalKmTime = 0.0f;
float totalMemcpyTime = 0.0f;
float totalFreeTime = 0.0f;

// Константы и параметры
const int size_mass = 100000;   // размерность массива
const int dim = 3;             // размерность пространства
const int k = 3;               // количество кластеров
const int max_iterations = 10; // Максимальное количество итераций
// const float epsilon = 0.00001; // Пороговое значение для остановки

// Путь к файлу с данными
string file = "my_mass100000.txt";

// Функция для считывания из файла данных с размерностью (100000, 3)
void readFromFile(float* data, int size, string fileName) {
    ifstream file(fileName);
    if (file.is_open()) {
        string line;
        int i = 0;
        while (getline(file, line) && i < size) {
            istringstream iss(line);
            float x, y, z;
            if (iss >> x >> y >> z) {
                data[i * dim] = x;
                data[i * dim + 1] = y;
                data[i * dim + 2] = z;
                i++;
            }
        }
        file.close();
    } else {
        cerr << "Error opening file: " << fileName << endl;
    }
}

// Задание начальных центров на CPU
void startingCenters(float* centers, float* data) {
    float minValues[dim], maxValues[dim];
    for (int i = 0; i < dim; i++) {
        minValues[i] = numeric_limits<float>::max();
        maxValues[i] = numeric_limits<float>::lowest();
    }

    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            centers[i * dim + j] = 0;
        }
    }

    for (int i = 0; i < size_mass; i++) {
        for (int j = 0; j < dim; j++) {
            minValues[j] = min(minValues[j], data[i * dim + j]);
            maxValues[j] = max(maxValues[j], data[i * dim + j]);
        }
    }

    random_device rd;
    mt19937 gen(rd());

    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            uniform_real_distribution<> dis(minValues[j], maxValues[j]);
            centers[i * dim + j] = dis(gen);
        }
    }
}

// Объявление константной памяти на GPU
__constant__ int d_k;
__constant__ int d_dim;
__constant__ int d_size_mass;

// CUDA-ядро для заполнения кластеров
__global__ void clusterDistrCUDA(float* data, float* centers, float* distances, int* cluster_indexes) {
    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    // Работа всех нитей
    if (point < d_size_mass) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            float sum = 0.0f;
            for (int coord = 0; coord < d_dim; coord++) {
                sum += powf((data[point * d_dim + coord] - centers[num_center * d_dim + coord]), 2);
            }
            float dist = sqrtf(sum);
            distances[point * d_k + num_center] = dist;
        }

        int min_index = 0;
        float min_distance = distances[point * d_k];

        for (int dist_id = 1; dist_id < d_k; dist_id++) {
            if (distances[point * d_k + dist_id] < min_distance) {
                min_distance = distances[point * d_k + dist_id];
                min_index = dist_id;
            }
        }

        cluster_indexes[point] = min_index;
    }
}

// CUDA-ядро для обнуления счётчика и центров
__global__ void nullCentersCUDA(float* data, int* cluster_indexes, float* centers, int* count_cluster_i) {
    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    // Блок для одной нити, обнуление счётчика
    if (point == 0) {
        for (int i = 0; i < d_k; i++) {
            for (int j = 0; j < d_dim; j++) {
                centers[i * d_dim + j] = 0.0f;
            }
            count_cluster_i[i] = 0;
        }
    }
}

// CUDA-ядро для пересчёта центров
__global__ void recalculCentersCUDA(float* data, int* cluster_indexes, float* centers, int* count_cluster_i) {
    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    // Блок для всех нитей
    if (point < d_size_mass) {
        int cluster_id = cluster_indexes[point];
        atomicAdd(&count_cluster_i[cluster_id], 1);
        for (int j = 0; j < d_dim; j++) {
            // если точка относится к кластеру, её координаты суммируются
            atomicAdd(&centers[cluster_id * d_dim + j], data[point * d_dim + j]);
        }
    }
}

__global__ void coordCUDA(float* data, int* cluster_indexes, float* centers, int* count_cluster_i) {
    int point = blockIdx.x * blockDim.x + threadIdx.x;
    // блок для одной нити
    if (point == 0) {
        for (int i = 0; i < d_k; i++) {
            for (int j = 0; j < d_dim; j++) {
                if (count_cluster_i[i] != 0) {
                    // вычисление координаты
                    centers[i * d_dim + j] /= count_cluster_i[i];
                }
            }
        }
    }
}

void k_means(float* data, int* cluster_indexes, float* centers, float* oldCenters, float* distances, float* dev_data, float* dev_centers, float* dev_distances, int* dev_cluster_indexes, int* dev_count_cluster_i) {
    // Копирование констант на GPU
    cudaMemcpyToSymbol(d_k, &k, sizeof(int));
    cudaMemcpyToSymbol(d_dim, &dim, sizeof(int));
    cudaMemcpyToSymbol(d_size_mass, &size_mass, sizeof(int));

    // Инициализация центров на CPU
    startingCenters(centers, data);

    // Перенос начальных центров на GPU
    cudaMemcpy(dev_centers, centers, k * dim * sizeof(float), cudaMemcpyHostToDevice);

    // Сохранение начальных центров на CPU
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            oldCenters[i * dim + j] = centers[i * dim + j];
        }
    }

    int iteration = 0;
    while (iteration < max_iterations) {
        // Определение параметров для запуска ядер
        int blockSize = 256;
        int gridSize = (size_mass + blockSize - 1) / blockSize;

        // запуск ядра для заполнения кластеров
        clusterDistrCUDA<<<gridSize, blockSize>>>(dev_data, dev_centers, dev_distances, dev_cluster_indexes);
        cudaDeviceSynchronize();

        nullCentersCUDA<<<1, 1>>>(dev_data, dev_cluster_indexes, dev_centers, dev_count_cluster_i);
        cudaDeviceSynchronize();

        // запуск ядра для пересчёта центров
        recalculCentersCUDA<<<gridSize, blockSize>>>(dev_data, dev_cluster_indexes, dev_centers, dev_count_cluster_i);
        cudaDeviceSynchronize();

        coordCUDA<<<1, 1>>>(dev_data, dev_cluster_indexes, dev_centers, dev_count_cluster_i);
        cudaDeviceSynchronize();

        // копирование пересчитанных центров с GPU на CPU
        cudaMemcpy(centers, dev_centers, k * dim * sizeof(float), cudaMemcpyDeviceToHost);

        // проверка сходимости
        // if (iteration > 0 && converged(centers, oldCenters, epsilon, k)) {
        //     cout << "The algorithm converged on iteration " << iteration << "." << endl << endl;
        //     break;
        // }

        // сохранение вычисленных центров
        for (int i = 0; i < k; i++) {
            for (int j = 0; j < dim; j++) {
                oldCenters[i * dim + j] = centers[i * dim + j];
            }
        }

        iteration++;
    }

    // Копирование меток для точек с GPU на CPU
    cudaMemcpy(cluster_indexes, dev_cluster_indexes, size_mass * sizeof(int), cudaMemcpyDeviceToHost);
}

int main() {

    for (int iter = 0; iter < iterations; iter++) {
        // Выделение памяти на CPU
        float* myArray = new float[size_mass * dim];
        float* centers = new float[k * dim];
        float* oldCenters = new float[k * dim];
        float* distances = new float[size_mass * k];
        int* cluster_indexes = new int[size_mass];

        // -------------------------------------------------------------------------
        // замеры времени выделения памяти с усреднением по итерациям
        cudaEvent_t start1, stop1;
        cudaEventCreate(&start1);
        cudaEventCreate(&stop1);

        cudaEventRecord(start1);

        // Выделение памяти на GPU
        float *dev_data, *dev_centers, *dev_distances;
        int *dev_cluster_indexes, *dev_count_cluster_i;

        cudaMalloc((void**)&dev_data, size_mass * dim * sizeof(float));
        cudaMalloc((void**)&dev_centers, k * dim * sizeof(float));
        cudaMalloc((void**)&dev_distances, size_mass * k * sizeof(float));
        cudaMalloc((void**)&dev_cluster_indexes, size_mass * sizeof(int));
        cudaMalloc((void**)&dev_count_cluster_i, k * sizeof(int));

        cudaEventRecord(stop1);
        cudaEventSynchronize(stop1);
        float mallocTime;
        cudaEventElapsedTime(&mallocTime, start1, stop1);
        totalMallocTime += mallocTime;
        cudaEventDestroy(start1);
        cudaEventDestroy(stop1);
        // --------------------------------------------------------------------------

        // Считывание данных из файла и запись в массив
        readFromFile(myArray, size_mass, file);

        // __________________
        // замеры времени копирования данных на устройство с усреднением по итерациям
        cudaEvent_t start2, stop2;
        cudaEventCreate(&start2);
        cudaEventCreate(&stop2);

        cudaEventRecord(start2);

        // Копирование данных на устройство
        cudaMemcpy(dev_data, myArray, size_mass * dim * sizeof(float), cudaMemcpyHostToDevice);

        cudaEventRecord(stop2);
        cudaEventSynchronize(stop2);
        float memcpyTime;
        cudaEventElapsedTime(&memcpyTime, start2, stop2);
        totalMemcpyTime += memcpyTime;
        cudaEventDestroy(start2);
        cudaEventDestroy(stop2);
        // __________________

        // Запуск алгоритма
        // cout << "K-means algorithm. Mass (100000,3)." << endl << endl;

        // ...........................................................................
        // замеры времени работы алгоритма K-means с усреднением по итерациям
        cudaEvent_t start3, stop3;
        cudaEventCreate(&start3);
        cudaEventCreate(&stop3);

        cudaEventRecord(start3);

        k_means(myArray, cluster_indexes, centers, oldCenters, distances, dev_data, dev_centers, dev_distances, dev_cluster_indexes, dev_count_cluster_i);

        cudaEventRecord(stop3);
        cudaEventSynchronize(stop3);
        float kmTime;
        cudaEventElapsedTime(&kmTime, start3, stop3);
        totalKmTime += kmTime;
        cudaEventDestroy(start3);
        cudaEventDestroy(stop3);
        // ............................................................................

        // Освобождение памяти на CPU
        delete[] myArray;
        delete[] centers;
        delete[] oldCenters;
        delete[] distances;
        delete[] cluster_indexes;

        // ******************
        // замеры времени освобождения памяти на GPU с усреднением по итерациям
        cudaEvent_t start4, stop4;
        cudaEventCreate(&start4);
        cudaEventCreate(&stop4);

        cudaEventRecord(start4);

        // Освобождение памяти на GPU
        cudaFree(dev_data);
        cudaFree(dev_centers);
        cudaFree(dev_distances);
        cudaFree(dev_cluster_indexes);
        cudaFree(dev_count_cluster_i);

        cudaEventRecord(stop4);
        cudaEventSynchronize(stop4);
        float freeTime;
        cudaEventElapsedTime(&freeTime, start4, stop4);
        totalFreeTime += freeTime;
        cudaEventDestroy(start4);
        cudaEventDestroy(stop4);
        // ****************
    }

    cout << "\nK_means. For mass 100000 points" << endl;
    cout << "GPU memory allocation time: " << totalMallocTime / iterations << " ms" << endl;
    cout << "Time to copy data from the host to the device: " << totalMemcpyTime / iterations << " ms" << endl;
    cout << "K-means: " << totalKmTime / iterations << " ms" << endl;
    cout << "Free GPU: " << totalFreeTime / iterations << " ms" << endl;
    cout << "Total: " << (totalMallocTime + totalMemcpyTime + totalKmTime + totalFreeTime) / iterations << " ms" << endl;

    return 0;
}

Writing K-means100000_CUDA.cu


In [ ]:
!nvcc K-means100000_CUDA.cu -o K-means100000_CUDA
# компилляция

In [ ]:
# Запуск
!./K-means100000_CUDA


K_means. For mass 100000 points
GPU memory allocation time: 0.217071 ms
Time to copy data from the host to the device: 0.3476 ms
K-means: 14.0445 ms
Free GPU: 0.336089 ms
Total: 14.9452 ms


In [ ]:
%%writefile K-means1000000_CUDA.cu

#include <iostream>
#include <random>
#include <fstream>
#include <sstream>
#include <cmath>
#include <algorithm>
#include <limits>
#include <cuda_runtime.h>

using namespace std;

// Константы для замера времени
const int iterations = 100;
float totalMallocTime = 0.0f;
float totalKmTime = 0.0f;
float totalMemcpyTime = 0.0f;
float totalFreeTime = 0.0f;

// Константы и параметры
const int size_mass = 1000000;   // размерность массива
const int dim = 3;             // размерность пространства
const int k = 3;               // количество кластеров
const int max_iterations = 10; // Максимальное количество итераций
// const float epsilon = 0.00001; // Пороговое значение для остановки

// Путь к файлу с данными
string file = "my_mass1000000.txt";

// Функция для считывания из файла данных с размерностью (1000000, 3)
void readFromFile(float* data, int size, string fileName) {
    ifstream file(fileName);
    if (file.is_open()) {
        string line;
        int i = 0;
        while (getline(file, line) && i < size) {
            istringstream iss(line);
            float x, y, z;
            if (iss >> x >> y >> z) {
                data[i * dim] = x;
                data[i * dim + 1] = y;
                data[i * dim + 2] = z;
                i++;
            }
        }
        file.close();
    } else {
        cerr << "Error opening file: " << fileName << endl;
    }
}


// Функция для считывания данных из файла размерностью (1000000, 3)
void readFromFile(float* data, int size, int part_number) {
    string filename = "my_mass1000000_part_" + to_string(part_number) + ".txt";
    ifstream file(filename);
    if (file.is_open()) {
        string line;
        int i = 0;
        while (getline(file, line) && i < size) {
            istringstream iss(line);
            float x, y, z;
            if (iss >> x >> y >> z) {
                data[i * dim] = x;
                data[i * dim + 1] = y;
                data[i * dim + 2] = z;
                i++;
            }
        }
        file.close();
    }
    else {
        cerr << "Error opening file: " << filename << endl;
    }
}

// Задание начальных центров на CPU
void startingCenters(float* centers, float* data) {
    float minValues[dim], maxValues[dim];
    for (int i = 0; i < dim; i++) {
        minValues[i] = numeric_limits<float>::max();
        maxValues[i] = numeric_limits<float>::lowest();
    }

    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            centers[i * dim + j] = 0;
        }
    }

    for (int i = 0; i < size_mass; i++) {
        for (int j = 0; j < dim; j++) {
            minValues[j] = min(minValues[j], data[i * dim + j]);
            maxValues[j] = max(maxValues[j], data[i * dim + j]);
        }
    }

    random_device rd;
    mt19937 gen(rd());

    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            uniform_real_distribution<> dis(minValues[j], maxValues[j]);
            centers[i * dim + j] = dis(gen);
        }
    }
}

// Объявление константной памяти на GPU
__constant__ int d_k;
__constant__ int d_dim;
__constant__ int d_size_mass;

// CUDA-ядро для заполнения кластеров
__global__ void clusterDistrCUDA(float* data, float* centers, float* distances, int* cluster_indexes) {
    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    // Работа всех нитей
    if (point < d_size_mass) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            float sum = 0.0f;
            for (int coord = 0; coord < d_dim; coord++) {
                sum += powf((data[point * d_dim + coord] - centers[num_center * d_dim + coord]), 2);
            }
            float dist = sqrtf(sum);
            distances[point * d_k + num_center] = dist;
        }

        int min_index = 0;
        float min_distance = distances[point * d_k];

        for (int dist_id = 1; dist_id < d_k; dist_id++) {
            if (distances[point * d_k + dist_id] < min_distance) {
                min_distance = distances[point * d_k + dist_id];
                min_index = dist_id;
            }
        }

        cluster_indexes[point] = min_index;
    }
}

// CUDA-ядро для обнуления счётчика и центров
__global__ void nullCentersCUDA(float* data, int* cluster_indexes, float* centers, int* count_cluster_i) {
    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    // Блок для одной нити, обнуление счётчика
    if (point == 0) {
        for (int i = 0; i < d_k; i++) {
            for (int j = 0; j < d_dim; j++) {
                centers[i * d_dim + j] = 0.0f;
            }
            count_cluster_i[i] = 0;
        }
    }
}

// CUDA-ядро для пересчёта центров
__global__ void recalculCentersCUDA(float* data, int* cluster_indexes, float* centers, int* count_cluster_i) {
    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    // Блок для всех нитей
    if (point < d_size_mass) {
        int cluster_id = cluster_indexes[point];
        atomicAdd(&count_cluster_i[cluster_id], 1);
        for (int j = 0; j < d_dim; j++) {
            // если точка относится к кластеру, её координаты суммируются
            atomicAdd(&centers[cluster_id * d_dim + j], data[point * d_dim + j]);
        }
    }
}

__global__ void coordCUDA(float* data, int* cluster_indexes, float* centers, int* count_cluster_i) {
    int point = blockIdx.x * blockDim.x + threadIdx.x;
    // блок для одной нити
    if (point == 0) {
        for (int i = 0; i < d_k; i++) {
            for (int j = 0; j < d_dim; j++) {
                if (count_cluster_i[i] != 0) {
                    // вычисление координаты
                    centers[i * d_dim + j] /= count_cluster_i[i];
                }
            }
        }
    }
}

void k_means(float* data, int* cluster_indexes, float* centers, float* oldCenters, float* distances, float* dev_data, float* dev_centers, float* dev_distances, int* dev_cluster_indexes, int* dev_count_cluster_i) {
    // Копирование констант на GPU
    cudaMemcpyToSymbol(d_k, &k, sizeof(int));
    cudaMemcpyToSymbol(d_dim, &dim, sizeof(int));
    cudaMemcpyToSymbol(d_size_mass, &size_mass, sizeof(int));

    // Инициализация центров на CPU
    startingCenters(centers, data);

    // Перенос начальных центров на GPU
    cudaMemcpy(dev_centers, centers, k * dim * sizeof(float), cudaMemcpyHostToDevice);

    // Сохранение начальных центров на CPU
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            oldCenters[i * dim + j] = centers[i * dim + j];
        }
    }

    int iteration = 0;
    while (iteration < max_iterations) {
        // Определение параметров для запуска ядер
        int blockSize = 256;
        int gridSize = (size_mass + blockSize - 1) / blockSize;

        // запуск ядра для заполнения кластеров
        clusterDistrCUDA<<<gridSize, blockSize>>>(dev_data, dev_centers, dev_distances, dev_cluster_indexes);
        cudaDeviceSynchronize();

        nullCentersCUDA<<<1, 1>>>(dev_data, dev_cluster_indexes, dev_centers, dev_count_cluster_i);
        cudaDeviceSynchronize();

        // запуск ядра для пересчёта центров
        recalculCentersCUDA<<<gridSize, blockSize>>>(dev_data, dev_cluster_indexes, dev_centers, dev_count_cluster_i);
        cudaDeviceSynchronize();

        coordCUDA<<<1, 1>>>(dev_data, dev_cluster_indexes, dev_centers, dev_count_cluster_i);
        cudaDeviceSynchronize();

        // копирование пересчитанных центров с GPU на CPU
        cudaMemcpy(centers, dev_centers, k * dim * sizeof(float), cudaMemcpyDeviceToHost);

        // проверка сходимости
        // if (iteration > 0 && converged(centers, oldCenters, epsilon, k)) {
        //     cout << "The algorithm converged on iteration " << iteration << "." << endl << endl;
        //     break;
        // }

        // сохранение вычисленных центров
        for (int i = 0; i < k; i++) {
            for (int j = 0; j < dim; j++) {
                oldCenters[i * dim + j] = centers[i * dim + j];
            }
        }

        iteration++;
    }

    // Копирование меток для точек с GPU на CPU
    cudaMemcpy(cluster_indexes, dev_cluster_indexes, size_mass * sizeof(int), cudaMemcpyDeviceToHost);
}

int main() {

    for (int iter = 0; iter < iterations; iter++) {
        // Выделение памяти на CPU
        float* myArray = new float[size_mass * dim];
        float* centers = new float[k * dim];
        float* oldCenters = new float[k * dim];
        float* distances = new float[size_mass * k];
        int* cluster_indexes = new int[size_mass];

        // -------------------------------------------------------------------------
        // замеры времени выделения памяти с усреднением по итерациям
        cudaEvent_t start1, stop1;
        cudaEventCreate(&start1);
        cudaEventCreate(&stop1);

        cudaEventRecord(start1);

        // Выделение памяти на GPU
        float *dev_data, *dev_centers, *dev_distances;
        int *dev_cluster_indexes, *dev_count_cluster_i;

        cudaMalloc((void**)&dev_data, size_mass * dim * sizeof(float));
        cudaMalloc((void**)&dev_centers, k * dim * sizeof(float));
        cudaMalloc((void**)&dev_distances, size_mass * k * sizeof(float));
        cudaMalloc((void**)&dev_cluster_indexes, size_mass * sizeof(int));
        cudaMalloc((void**)&dev_count_cluster_i, k * sizeof(int));

        cudaEventRecord(stop1);
        cudaEventSynchronize(stop1);
        float mallocTime;
        cudaEventElapsedTime(&mallocTime, start1, stop1);
        totalMallocTime += mallocTime;
        cudaEventDestroy(start1);
        cudaEventDestroy(stop1);
        // --------------------------------------------------------------------------

        // Считывание данных (1000000, 3) из файла и запись в массив
        for (int i = 0; i <= 9; i++) {
            readFromFile(myArray + i * 100000, size_mass / 10, i);
        }


        // __________________
        // замеры времени копирования данных на устройство с усреднением по итерациям
        cudaEvent_t start2, stop2;
        cudaEventCreate(&start2);
        cudaEventCreate(&stop2);

        cudaEventRecord(start2);

        // Копирование данных на устройство
        cudaMemcpy(dev_data, myArray, size_mass * dim * sizeof(float), cudaMemcpyHostToDevice);

        cudaEventRecord(stop2);
        cudaEventSynchronize(stop2);
        float memcpyTime;
        cudaEventElapsedTime(&memcpyTime, start2, stop2);
        totalMemcpyTime += memcpyTime;
        cudaEventDestroy(start2);
        cudaEventDestroy(stop2);
        // __________________

        // Запуск алгоритма
        // cout << "K-means algorithm. Mass (1000000,3)." << endl << endl;

        // ...........................................................................
        // замеры времени работы алгоритма K-means с усреднением по итерациям
        cudaEvent_t start3, stop3;
        cudaEventCreate(&start3);
        cudaEventCreate(&stop3);

        cudaEventRecord(start3);

        k_means(myArray, cluster_indexes, centers, oldCenters, distances, dev_data, dev_centers, dev_distances, dev_cluster_indexes, dev_count_cluster_i);

        cudaEventRecord(stop3);
        cudaEventSynchronize(stop3);
        float kmTime;
        cudaEventElapsedTime(&kmTime, start3, stop3);
        totalKmTime += kmTime;
        cudaEventDestroy(start3);
        cudaEventDestroy(stop3);
        // ............................................................................

        // Освобождение памяти на CPU
        delete[] myArray;
        delete[] centers;
        delete[] oldCenters;
        delete[] distances;
        delete[] cluster_indexes;

        // ******************
        // замеры времени освобождения памяти на GPU с усреднением по итерациям
        cudaEvent_t start4, stop4;
        cudaEventCreate(&start4);
        cudaEventCreate(&stop4);

        cudaEventRecord(start4);

        // Освобождение памяти на GPU
        cudaFree(dev_data);
        cudaFree(dev_centers);
        cudaFree(dev_distances);
        cudaFree(dev_cluster_indexes);
        cudaFree(dev_count_cluster_i);

        cudaEventRecord(stop4);
        cudaEventSynchronize(stop4);
        float freeTime;
        cudaEventElapsedTime(&freeTime, start4, stop4);
        totalFreeTime += freeTime;
        cudaEventDestroy(start4);
        cudaEventDestroy(stop4);
        // ****************
    }

    cout << "\nK_means. For mass 1000000 points" << endl;
    cout << "GPU memory allocation time: " << totalMallocTime / iterations << " ms" << endl;
    cout << "Time to copy data from the host to the device: " << totalMemcpyTime / iterations << " ms" << endl;
    cout << "K-means: " << totalKmTime / iterations << " ms" << endl;
    cout << "Free GPU: " << totalFreeTime / iterations << " ms" << endl;
    cout << "Total: " << (totalMallocTime + totalMemcpyTime + totalKmTime + totalFreeTime) / iterations << " ms" << endl;

    return 0;
}

Writing K-means1000000_CUDA.cu


In [ ]:
!nvcc K-means1000000_CUDA.cu -o K-means1000000_CUDA
# компилляция

In [ ]:
# Запуск
!./K-means1000000_CUDA


K_means. For mass 1000000 points
GPU memory allocation time: 0.376005 ms
Time to copy data from the host to the device: 2.23772 ms
K-means: 133.978 ms
Free GPU: 1.41535 ms
Total: 138.007 ms


In [ ]:
%%writefile C-means100000_CUDA.cu
#include <iostream>
#include <random>
#include <fstream>
#include <sstream>
#include <cmath>
#include <limits>
#include <cuda_runtime.h>

using namespace std;

// Константы для замера времени
const int iterations = 10;
float totalMallocTime = 0.0f;
float totalCmTime = 0.0f;
float totalMemcpyTime = 0.0f;
float totalFreeTime = 0.0f;

// Размерность массива
const int size_mass = 100000;
const int dim = 3; // размерность пространства
const int k = 3; // количество кластеров

const int m = 2; // Экспоненциальный вес
const int max_iterations = 10; // Максимальное количество итераций
//const float epsilon = 0.0001; // Пороговое значение для остановки

// Объявление константной памяти на GPU
__constant__ int d_k;
__constant__ int d_dim;
__constant__ int d_m;
__constant__ int d_size_mass;

// Путь к файлу с данными
string file = "my_mass100000.txt";

// Функция для считывания данных из файла
void readFromFile(float* data, string fileName) {
    ifstream file(fileName);
    if (file.is_open()) {
        string line;
        int i = 0;
        while (getline(file, line) && i < size_mass) {
            istringstream iss(line);
            float x, y, z;
            if (iss >> x >> y >> z) {
                data[i * dim] = x;
                data[i * dim + 1] = y;
                data[i * dim + 2] = z;
                i++;
            }
        }
        file.close();
    } else {
        cerr << "Error opening file: " << fileName << endl;
    }
}

// на CPU
void startingCenters(float* centers, float* data) {
    float minValues[dim];
    float maxValues[dim];

    for (int i = 0; i < dim; i++) {
        minValues[i] = numeric_limits<float>::max();
        maxValues[i] = numeric_limits<float>::lowest();
    }

    for (int i = 0; i < size_mass; i++) {
        for (int j = 0; j < dim; j++) {
            minValues[j] = min(minValues[j], data[i * dim + j]);
            maxValues[j] = max(maxValues[j], data[i * dim + j]);
        }
    }

    random_device rd;
    mt19937 gen(rd());

    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            uniform_real_distribution<> dis(minValues[j], maxValues[j]);
            centers[i * dim + j] = dis(gen);
        }
    }
}

// на GPU

// Пересчёт матрицы степеней принадлежности на основе вычисленных центроид
__global__ void updateMembershipCUDA(float* dataPoints, float* centers, float* distances, float* membership) {

    // Вычисление евклидовых расстояний от каждой точки до каждого центра

    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    if (point < d_size_mass) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            float sum = 0.0f;
            for (int coord = 0; coord < d_dim; coord++) {
                sum += powf((dataPoints[point * d_dim + coord] - centers[num_center * d_dim + coord]), 2);
            }
            float dist = sqrtf(sum);
            distances[point * d_k + num_center] = dist;
        }

        // Вычисление степеней принадлежности для каждой точки
        for (int j = 0; j < d_k; j++) {
            float sumDistances = 0.0f;
            for (int c = 0; c < d_k; c++) {
                float distancesRatio = distances[point * d_k + j] / distances[point * d_k + c];
                sumDistances += powf(distancesRatio, (2.0f / (d_m - 1)));
            }
            membership[point * d_k + j] = 1.0f / sumDistances;
        }
    }
}

// Функция для обнуления переменных
__global__ void updateCentersCUDA_one(float* data, float* centers, float* membership, float* numerator, float* denominator) {

    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    if (point == 0) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            for (int j = 0; j < d_dim; j++) {
                numerator[num_center * d_dim + j] = 0.0f;
            }
            denominator[num_center] = 0.0f;
        }
    }
}

// Функция для пересчёта центров
__global__ void updateCentersCUDA_two(float* data, float* centers, float* membership, float* numerator, float* denominator) {

    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;
    if (point < d_size_mass) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            float membershipPow = powf(membership[point * d_k + num_center], d_m);
            for (int j = 0; j < d_dim; j++) {
                atomicAdd(&numerator[num_center * d_dim + j], membershipPow * data[point * d_dim + j]);
            }
            atomicAdd(&denominator[num_center], membershipPow);
        }
    }
}

// Функция для пересчёта центров
__global__ void updateCentersCUDA_three(float* data, float* centers, float* membership, float* numerator, float* denominator) {

    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    if (point == 0) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            for (int j = 0; j < d_dim; j++) {
                centers[num_center * d_dim + j] = numerator[num_center * d_dim + j] / denominator[num_center];
            }
        }
    }
}

// Функция для проверки сходимости алгоритма
//bool converged(float* new_centers, float* old_centers) {
//    for (int i = 0; i < k; i++) {
//        for (int j = 0; j < dim; j++) {
//            if (fabs(new_centers[i * dim + j] - old_centers[i * dim + j]) >= epsilon) {
//                return false;
//            }
//        }
//    }
//    return true;
//}

// Основная функция алгоритма C-Means
void c_means(float* data, float* centers, float* oldCenters, float* membership, float* distances, float *dev_data, float *dev_centers, float *dev_distances, float *dev_membership, float* dev_numerator, float* dev_denominator) {

    // Копирование констант с CPU на GPU
    cudaMemcpyToSymbol(d_k, &k, sizeof(int));
    cudaMemcpyToSymbol(d_dim, &dim, sizeof(int));
    cudaMemcpyToSymbol(d_m, &m, sizeof(int));
    cudaMemcpyToSymbol(d_size_mass, &size_mass, sizeof(int));

    // Инициализация начальных центроид на CPU
    startingCenters(centers, data);

    // Перенос начальных центров на GPU
    cudaMemcpy(dev_centers, centers, k * dim * sizeof(float), cudaMemcpyHostToDevice);

    // Копирование текущих центров в массив oldCenters
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            oldCenters[i * dim + j] = centers[i * dim + j];
        }
    }

    // Итерационный процесс
    int iteration = 0;
    while (iteration < max_iterations) {

        // Определение параметров для запуска ядер
        int blockSize = 256;
        int gridSize = (size_mass * dim + blockSize - 1) / blockSize;

        // Запуск ядра для расчёта матрицы степеней принадлежности на основе текущих центров
        updateMembershipCUDA<<<gridSize, blockSize>>>(dev_data, dev_centers, dev_distances, dev_membership);
        cudaDeviceSynchronize();

        // Запуск ядер для пересчёта центров на основе текущей матрицы степеней принадлежности
        updateCentersCUDA_one<<<1, 1>>>(dev_data, dev_centers, dev_membership, dev_numerator, dev_denominator);
        cudaDeviceSynchronize();

        updateCentersCUDA_two<<<gridSize, blockSize>>>(dev_data, dev_centers, dev_membership, dev_numerator, dev_denominator);
        cudaDeviceSynchronize();

        updateCentersCUDA_three<<<1, 1>>>(dev_data, dev_centers, dev_membership, dev_numerator, dev_denominator);
        cudaDeviceSynchronize();

        // Копирование новых центров с GPU на CPU
        cudaMemcpy(centers, dev_centers, k * dim * sizeof(float), cudaMemcpyDeviceToHost);

        // Условие остановки алгоритма
//        if (converged(centers, oldCenters)) {
//            cout << "The algorithm converged on iteration " << iteration << "." << endl << endl;
//            break;
//        }

        // Обновление центров
        for (int i = 0; i < k; i++) {
            for (int j = 0; j < dim; j++) {
                oldCenters[i * dim + j] = centers[i * dim + j];
            }
        }

        iteration++;
    }



int main() {


// Замеры времени для iterations запусков
 for (int iter = 0; iter < iterations; iter++) {


    float* data = new float[size_mass * dim];
    float* centers = new float[k * dim];
    float* oldCenters = new float[k * dim];
    float* membership = new float[size_mass * k];
    float* distances = new float[size_mass * k];

    // Считывание данных из файла
    readFromFile(data, file);


        // -------------------------------------------------------------------------
        // замеры времени выделения памяти с усреднением по итерациям
        cudaEvent_t start1, stop1;
        cudaEventCreate(&start1);
        cudaEventCreate(&stop1);

        cudaEventRecord(start1);

    // Указатели на GPU память
    float* dev_data;
    float* dev_centers;
    float* dev_distances;
    float* dev_membership;
    float* dev_numerator;
    float* dev_denominator;

    // Выделение памяти на GPU
    cudaMalloc((void**)&dev_data, size_mass * dim * sizeof(float));
    cudaMalloc((void**)&dev_centers, k * dim * sizeof(float));
    cudaMalloc((void**)&dev_distances, size_mass * k * sizeof(float));
    cudaMalloc((void**)&dev_membership, size_mass * k * sizeof(float));
    cudaMalloc((void**)&dev_numerator, k * dim * sizeof(float));
    cudaMalloc((void**)&dev_denominator, k * sizeof(float));

        cudaEventRecord(stop1);
        cudaEventSynchronize(stop1);
        float mallocTime;
        cudaEventElapsedTime(&mallocTime, start1, stop1);
        totalMallocTime += mallocTime;
        cudaEventDestroy(start1);
        cudaEventDestroy(stop1);
        // --------------------------------------------------------------------------


        // __________________
        // замеры времени копирования данных на устройство с усреднением по итерациям
        cudaEvent_t start2, stop2;
        cudaEventCreate(&start2);
        cudaEventCreate(&stop2);

        cudaEventRecord(start2);

    // Копирование данных с CPU на GPU
    cudaMemcpy(dev_data, data, size_mass * dim * sizeof(float), cudaMemcpyHostToDevice);


        cudaEventRecord(stop2);
        cudaEventSynchronize(stop2);
        float memcpyTime;
        cudaEventElapsedTime(&memcpyTime, start2, stop2);
        totalMemcpyTime += memcpyTime;
        cudaEventDestroy(start2);
        cudaEventDestroy(stop2);
        // __________________

        // ...........................................................................
        // замеры времени работы алгоритма K-means с усреднением по итерациям
        cudaEvent_t start3, stop3;
        cudaEventCreate(&start3);
        cudaEventCreate(&stop3);

        cudaEventRecord(start3);


    // Запуск алгоритма C-Means
    c_means(data, centers, oldCenters, membership, distances, dev_data, dev_centers, dev_distances, dev_membership, dev_numerator, dev_denominator);

        cudaEventRecord(stop3);
        cudaEventSynchronize(stop3);
        float cmTime;
        cudaEventElapsedTime(&cmTime, start3, stop3);
        totaCmTime += cmTime;
        cudaEventDestroy(start3);
        cudaEventDestroy(stop3);
        // ............................................................................


        // ******************
        // замеры времени освобождения памяти на GPU с усреднением по итерациям
        cudaEvent_t start4, stop4;
        cudaEventCreate(&start4);
        cudaEventCreate(&stop4);

        cudaEventRecord(start4);


    // Освобождение памяти на GPU
    cudaFree(dev_data);
    cudaFree(dev_centers);
    cudaFree(dev_distances);
    cudaFree(dev_membership);
    cudaFree(dev_numerator);
    cudaFree(dev_denominator);

        cudaEventRecord(stop4);
        cudaEventSynchronize(stop4);
        float freeTime;
        cudaEventElapsedTime(&freeTime, start4, stop4);
        totalFreeTime += freeTime;
        cudaEventDestroy(start4);
        cudaEventDestroy(stop4);
        // ****************

    // Освобождение памяти на CPU
    delete[] data;
    delete[] centers;
    delete[] oldCenters;
    delete[] membership;
    delete[] distances;
}

// конец замеров и запусков


    cout << "\nC_means. For mass 100000 points" << endl;
    cout << "GPU memory allocation time: " << totalMallocTime / iterations << " ms" << endl;
    cout << "Time to copy data from the host to the device: " << totalMemcpyTime / iterations << " ms" << endl;
    cout << "C-means: " << totalCmTime / iterations << " ms" << endl;
    cout << "Free GPU: " << totalFreeTime / iterations << " ms" << endl;
    cout << "Total: " << (totalMallocTime + totalMemcpyTime + totalKmTime + totalFreeTime) / iterations << " ms" << endl;


    return 0;
}


Overwriting C-means100000_CUDA.cu


In [ ]:
%%writefile C-means10000_CUDA.cu

#include <iostream>
#include <random>
#include <fstream>
#include <sstream>
#include <cmath>
#include <limits>
#include <cuda_runtime.h>

using namespace std;

// Константы для замера времени
const int iterations = 10;
float totalMallocTime = 0.0f;
float totalCmTime = 0.0f;
float totalMemcpyTime = 0.0f;
float totalFreeTime = 0.0f;

// Размерность массива
const int size_mass = 10000;
const int dim = 3; // размерность пространства
const int k = 3; // количество кластеров

const int m = 2; // Экспоненциальный вес
const int max_iterations = 10; // Максимальное количество итераций
const float epsilon = 0.0001; // Пороговое значение для остановки

// Объявление константной памяти на GPU
__constant__ int d_k;
__constant__ int d_dim;
__constant__ int d_m;
__constant__ int d_size_mass;

// Путь к файлу с данными
string file = "my_mass10000.txt";



// Функция для считывания данных из файла
void readFromFile(float* data, string fileName) {
    ifstream file(fileName);
    if (file.is_open()) {
        string line;
        int i = 0;
        while (getline(file, line) && i < size_mass) {
            istringstream iss(line);
            float x, y, z;
            if (iss >> x >> y >> z) {
                data[i * dim] = x;
                data[i * dim + 1] = y;
                data[i * dim + 2] = z;
                i++;
            }
        }
        file.close();
    } else {
        cerr << "Error opening file: " << fileName << endl;
    }
}

// на CPU
void startingCenters(float* centers, float* data) {
    float minValues[dim];
    float maxValues[dim];

    for (int i = 0; i < dim; i++) {
        minValues[i] = numeric_limits<float>::max();
        maxValues[i] = numeric_limits<float>::lowest();
    }

    for (int i = 0; i < size_mass; i++) {
        for (int j = 0; j < dim; j++) {
            minValues[j] = min(minValues[j], data[i * dim + j]);
            maxValues[j] = max(maxValues[j], data[i * dim + j]);
        }
    }

    random_device rd;
    mt19937 gen(rd());

    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            uniform_real_distribution<> dis(minValues[j], maxValues[j]);
            centers[i * dim + j] = dis(gen);
        }
    }
}

// на GPU

// Пересчёт матрицы степеней принадлежности на основе вычисленных центроид
__global__ void updateMembershipCUDA(float* dataPoints, float* centers, float* distances, float* membership) {

    // Вычисление евклидовых расстояний от каждой точки до каждого центра

    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    if (point < d_size_mass) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            float sum = 0.0f;
            for (int coord = 0; coord < d_dim; coord++) {
                sum += powf((dataPoints[point * d_dim + coord] - centers[num_center * d_dim + coord]), 2);
            }
            float dist = sqrtf(sum);
            distances[point * d_k + num_center] = dist;
        }

        // Вычисление степеней принадлежности для каждой точки
        for (int j = 0; j < d_k; j++) {
            float sumDistances = 0.0f;
            for (int c = 0; c < d_k; c++) {
                float distancesRatio = distances[point * d_k + j] / distances[point * d_k + c];
                sumDistances += powf(distancesRatio, (2.0f / (d_m - 1)));
            }
            membership[point * d_k + j] = 1.0f / sumDistances;
        }
    }
}

// Функция для обнуления переменных
__global__ void updateCentersCUDA_one(float* data, float* centers, float* membership, float* numerator, float* denominator) {

    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    if (point == 0) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            for (int j = 0; j < d_dim; j++) {
                numerator[num_center * d_dim + j] = 0.0f;
            }
            denominator[num_center] = 0.0f;
        }
    }
}

// Функция для пересчёта центров
__global__ void updateCentersCUDA_two(float* data, float* centers, float* membership, float* numerator, float* denominator) {

    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;
    if (point < d_size_mass) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            float membershipPow = powf(membership[point * d_k + num_center], d_m);
            for (int j = 0; j < d_dim; j++) {
                atomicAdd(&numerator[num_center * d_dim + j], membershipPow * data[point * d_dim + j]);
            }
            atomicAdd(&denominator[num_center], membershipPow);
        }
    }
}

// Функция для пересчёта центров
__global__ void updateCentersCUDA_three(float* data, float* centers, float* membership, float* numerator, float* denominator) {

    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    if (point == 0) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            for (int j = 0; j < d_dim; j++) {
                centers[num_center * d_dim + j] = numerator[num_center * d_dim + j] / denominator[num_center];
            }
        }
    }
}

// Функция для проверки сходимости алгоритма
bool converged(float* new_centers, float* old_centers) {
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            if (fabs(new_centers[i * dim + j] - old_centers[i * dim + j]) >= epsilon) {
                return false;
            }
        }
    }
    return true;
}

// Основная функция алгоритма C-Means
void c_means(float* data, float* centers, float* oldCenters, float* membership, float* distances, float *dev_data, float *dev_centers, float *dev_distances, float *dev_membership, float* dev_numerator, float* dev_denominator) {

    // Копирование констант с CPU на GPU
    cudaMemcpyToSymbol(d_k, &k, sizeof(int));
    cudaMemcpyToSymbol(d_dim, &dim, sizeof(int));
    cudaMemcpyToSymbol(d_m, &m, sizeof(int));
    cudaMemcpyToSymbol(d_size_mass, &size_mass, sizeof(int));

    // Инициализация начальных центроид на CPU
    startingCenters(centers, data);

    // Перенос начальных центров на GPU
    cudaMemcpy(dev_centers, centers, k * dim * sizeof(float), cudaMemcpyHostToDevice);

    // Копирование текущих центров в массив oldCenters
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            oldCenters[i * dim + j] = centers[i * dim + j];
        }
    }

    // Итерационный процесс
    int iteration = 0;
    while (iteration < max_iterations) {

        // Определение параметров для запуска ядер
        int blockSize = 256;
        int gridSize = (size_mass * dim + blockSize - 1) / blockSize;

        // Запуск ядра для расчёта матрицы степеней принадлежности на основе текущих центров
        updateMembershipCUDA<<<gridSize, blockSize>>>(dev_data, dev_centers, dev_distances, dev_membership);
        cudaDeviceSynchronize();

// Запуск ядер для пересчёта центров на основе текущей матрицы степеней принадлежности
        updateCentersCUDA_one<<<1, 1>>>(dev_data, dev_centers, dev_membership, dev_numerator, dev_denominator);
        cudaDeviceSynchronize();

        updateCentersCUDA_two<<<gridSize, blockSize>>>(dev_data, dev_centers, dev_membership, dev_numerator, dev_denominator);
        cudaDeviceSynchronize();

        updateCentersCUDA_three<<<1, 1>>>(dev_data, dev_centers, dev_membership, dev_numerator, dev_denominator);
        cudaDeviceSynchronize();

        // Копирование новых центров с GPU на CPU
        cudaMemcpy(centers, dev_centers, k * dim * sizeof(float), cudaMemcpyDeviceToHost);

        // Условие остановки алгоритма
   //     if (converged(centers, oldCenters)) {
   //         cout << "The algorithm converged on iteration " << iteration << "." << endl << endl;
   //         break;
   //     }

        // Обновление центров
        for (int i = 0; i < k; i++) {
            for (int j = 0; j < dim; j++) {
                oldCenters[i * dim + j] = centers[i * dim + j];
            }
        }

        iteration++;
    }
}

int main() {

    // Замеры времени для iterations запусков
    for (int iter = 0; iter < iterations; iter++) {

        float* data = new float[size_mass * dim];
        float* centers = new float[k * dim];
        float* oldCenters = new float[k * dim];
        float* membership = new float[size_mass * k];
        float* distances = new float[size_mass * k];

        // Считывание данных из файла
        readFromFile(data, file);

        // -------------------------------------------------------------------------
        // замеры времени выделения памяти с усреднением по итерациям
        cudaEvent_t start1, stop1;
        cudaEventCreate(&start1);
        cudaEventCreate(&stop1);

        cudaEventRecord(start1);

        // Указатели на GPU память
        float* dev_data;
        float* dev_centers;
        float* dev_distances;
        float* dev_membership;
        float* dev_numerator;
        float* dev_denominator;

        // Выделение памяти на GPU
        cudaMalloc((void**)&dev_data, size_mass * dim * sizeof(float));
        cudaMalloc((void**)&dev_centers, k * dim * sizeof(float));
        cudaMalloc((void**)&dev_distances, size_mass * k * sizeof(float));
        cudaMalloc((void**)&dev_membership, size_mass * k * sizeof(float));
        cudaMalloc((void**)&dev_numerator, k * dim * sizeof(float));
        cudaMalloc((void**)&dev_denominator, k * sizeof(float));

        cudaEventRecord(stop1);
        cudaEventSynchronize(stop1);
        float mallocTime;
        cudaEventElapsedTime(&mallocTime, start1, stop1);
        totalMallocTime += mallocTime;
        cudaEventDestroy(start1);
        cudaEventDestroy(stop1);
        // --------------------------------------------------------------------------

        // ______
        // замеры времени копирования данных на устройство с усреднением по итерациям
        cudaEvent_t start2, stop2;
        cudaEventCreate(&start2);
        cudaEventCreate(&stop2);

        cudaEventRecord(start2);

        // Копирование данных с CPU на GPU
        cudaMemcpy(dev_data, data, size_mass * dim * sizeof(float), cudaMemcpyHostToDevice);

        cudaEventRecord(stop2);
        cudaEventSynchronize(stop2);
        float memcpyTime;
        cudaEventElapsedTime(&memcpyTime, start2, stop2);
        totalMemcpyTime += memcpyTime;
        cudaEventDestroy(start2);
        cudaEventDestroy(stop2);
        // ______

        // ...........................................................................
        // замеры времени работы алгоритма K-means с усреднением по итерациям
        cudaEvent_t start3, stop3;
        cudaEventCreate(&start3);
        cudaEventCreate(&stop3);

        cudaEventRecord(start3);

        // Запуск алгоритма C-Means
        c_means(data, centers, oldCenters, membership, distances, dev_data, dev_centers, dev_distances, dev_membership, dev_numerator, dev_denominator);

        cudaEventRecord(stop3);
        cudaEventSynchronize(stop3);
        float cmTime;
        cudaEventElapsedTime(&cmTime, start3, stop3);
        totalCmTime += cmTime;
        cudaEventDestroy(start3);
        cudaEventDestroy(stop3);
        // ............................................................................

        // ******
        // замеры времени освобождения памяти на GPU с усреднением по итерациям
        cudaEvent_t start4, stop4;
        cudaEventCreate(&start4);
        cudaEventCreate(&stop4);

        cudaEventRecord(start4);

        // Освобождение памяти на GPU
        cudaFree(dev_data);
        cudaFree(dev_centers);
        cudaFree(dev_distances);
        cudaFree(dev_membership);
        cudaFree(dev_numerator);
        cudaFree(dev_denominator);

        cudaEventRecord(stop4);
        cudaEventSynchronize(stop4);
        float freeTime;
        cudaEventElapsedTime(&freeTime, start4, stop4);
        totalFreeTime += freeTime;
        cudaEventDestroy(start4);
        cudaEventDestroy(stop4);
        // ****

        // Освобождение памяти на CPU
        delete[] data;
        delete[] centers;
        delete[] oldCenters;
        delete[] membership;
        delete[] distances;
    }

    // конец замеров и запусков
    cout << "\nC_means. For mass 10000 points" << endl;
    cout << "GPU memory allocation time: " << totalMallocTime / iterations << " ms" << endl;
    cout << "Time to copy data from the host to the device: " << totalMemcpyTime / iterations << " ms" << endl;
    cout << "C-means: " << totalCmTime / iterations << " ms" << endl;
    cout << "Free GPU: " << totalFreeTime / iterations << " ms" << endl;
    cout << "Total: " << (totalMallocTime + totalMemcpyTime + totalCmTime + totalFreeTime) / iterations << " ms" << endl;

    return 0;
}







Overwriting C-means10000_CUDA.cu


In [ ]:
!nvcc C-means10000_CUDA.cu -o C-means10000_CUDA

In [ ]:
!./C-means10000_CUDA


C_means. For mass 10000 points
GPU memory allocation time: 0.140691 ms
Time to copy data from the host to the device: 0.047632 ms
C-means: 4.42579 ms
Free GPU: 0.127661 ms
Total: 4.74177 ms
